### IMPORTS

In [23]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [24]:
from num2words import num2words

In [64]:
pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 KB 1.0 MB/s eta 0:00:00 kB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=c7e899214ddac1d40f53e9fdddf952906fdb104c2ec6c61ec87f3fc9d7a02364
  Stored in directory: /home/sct/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


In [58]:
pip install inflect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 219.1 kB/s eta 0:00:00m eta -:--:--
Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 KB 72.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 KB 227.3 kB/s eta 0:00:00 kB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.3 MB/s eta 0:00:000:00:01m eta 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 1.2 MB/s eta 0:00:009 kB/s eta 0:00:0101
Note: you may need to restart the kernel to use updated packages.


In [13]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/sct/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /home/sct/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/sct/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/sct/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /home/sct/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /home/sct/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/sct/nltk_

True

### Read data

In [1]:
import pandas as pd

nr_source_df = pd.read_csv('non_rumor_source_tweet_data.csv')
nr_source_df.info()

r_source_df = pd.read_csv('rumor_source_tweet_data.csv')
r_source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         420 non-null    int64 
 1   tweet_id        420 non-null    int64 
 2   tweet_text      420 non-null    object
 3   favorite_count  420 non-null    int64 
 4   retweet_count   420 non-null    int64 
 5   created_at      420 non-null    object
 6   label           420 non-null    object
dtypes: int64(4), object(3)
memory usage: 23.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         470 non-null    int64 
 1   tweet_id        470 non-null    int64 
 2   tweet_text      470 non-null    object
 3   favorite_count  470 non-null    int64 
 4   retweet_count   470 non-null    int64 
 5   created_at      470 non-null    

In [2]:
text = nr_source_df['tweet_text']
label = nr_source_df['label']
data = {'text':text, 'label':label}
nr_data = pd.DataFrame(data)
nr_data.head()

,text,label
0,Ottawa police are confirming a shooting at the...,non-rumor
1,Passersby working on shooting victim. From the...,non-rumor
2,Shooting at the war memorial in Ottawa. http:/...,non-rumor
3,After today we will officially be done shootin...,non-rumor
4,KEY: Nothing in Wilson's story in new autopsy ...,non-rumor


In [3]:
text = r_source_df['tweet_text']
label = r_source_df['label']
data = {'text':text, 'label':label}
r_data = pd.DataFrame(data)
r_data.head()

,text,label
0,Recap: Gunman shot dead inside Parliament buil...,rumor
1,Soldier killed at war memorial identified as N...,rumor
2,Soldier killed at war memorial identified as C...,rumor
3,BREAKING: Two or three gunmen were involved in...,rumor
4,All 3 patients injured in #OttawaShooting rele...,rumor


In [4]:
#combine datasets of rumors and non-rumors
frames = [nr_source_df,r_source_df]
df = pd.concat(frames)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 890 entries, 0 to 469
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         890 non-null    int64 
 1   tweet_id        890 non-null    int64 
 2   tweet_text      890 non-null    object
 3   favorite_count  890 non-null    int64 
 4   retweet_count   890 non-null    int64 
 5   created_at      890 non-null    object
 6   label           890 non-null    object
dtypes: int64(4), object(3)
memory usage: 55.6+ KB


In [20]:
#select the text and labels for conducting preprocessing
text = df['tweet_text']
label = df['label']
data = {'text':text, 'label':label}
df = pd.DataFrame(data)
df.head()

,text,label
0,Ottawa police are confirming a shooting at the...,non-rumor
1,Passersby working on shooting victim. From the...,non-rumor
2,Shooting at the war memorial in Ottawa. http:/...,non-rumor
3,After today we will officially be done shootin...,non-rumor
4,KEY: Nothing in Wilson's story in new autopsy ...,non-rumor


In [57]:
# shuffle the DataFrame rows
from sklearn.utils import shuffle

df = shuffle(df).reset_index(drop=True)
df.head(10)

,text,label
0,.@ctvottawa confirms there were 3 separate sho...,rumor
1,Scene at Ottawa war memorial. Soldier on duty ...,rumor
2,Dramatic photos from @wcuddington on the scene...,non-rumor
3,"Canadian cabinet minister Jason Kenney says ""C...",non-rumor
4,"Our phone lines are open 24/7 for your use, if...",non-rumor
5,MORE: Canadian soldier shot at war memorial ha...,rumor
6,MORE: A spokesman says Prime Minister Stephen ...,non-rumor
7,Security bulletin from Parliament Hill: #ottaw...,non-rumor
8,"Sad about #OttawaShooting. Worse, shooter migh...",non-rumor
9,#RCMP news conference on #Ottawa shootings exp...,rumor


### Upper to Lowercase Transformation

In [8]:
def to_lowercase(df):
    df['text'] = df['text'].apply(lambda x: x.lower() if isinstance(x, str) else x)
    return df

### Stopword removal

In [9]:
#function to remove stopwords
def remove_stopwords(df):
    stop_words = set(stopwords.words('english'))
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
    return df

### Remove URL

In [10]:
def remove_URL(df):
    """Remove URLs from a sample string"""
    df['text'] = df['text'].apply(lambda x:re.sub(r"http\S+", "", x))
    return df

In [14]:
df = remove_URL(df)

### Remove Non-ASCII words

In [11]:
def remove_non_ascii(df):
    df['text'] = df['text'].apply(lambda x:' '.join([unicodedata.normalize('NFKD', x).encode('ascii', 'ignore')
                                                     .decode('utf-8', 'ignore') for words in x.split()]))
    return df

### Remove contractions

In [12]:
def replace_contractions(df):
    """Replace contractions in string of text"""
    df['text'] = df['text'].apply(lambda x:contractions.fix(x))
    return df

### Remove puntuation

In [13]:
def remove_punctuation(df):
    pattern = r'\'\S+'
    df['text'] = df['text'] .apply(lambda x: re.sub(pattern,"", str(x)))
    return df

### Replace numbers

In [14]:
def replace_numbers(df):
    p = inflect.engine()
    df['text'] = df['text'].apply(lambda text: ' '.join(num2words(int(word)) if word.isdigit() else word for word in text.split()))
    return df

### Lemmatization

In [15]:
def lemmatize_sentence(input_string):
    # Perform lemmatization
    words = word_tokenize(input_string)
    # print("{0:20}{1:20}".format("--Word--", "--Lemma--"))
    lemmatized_sentence = reduce(lambda x, y: x + " " + wnl.lemmatize(y, pos="v"), words, "")
    return lemmatized_sentence

In [16]:
def lemmatize(df):
    try:
        for i in df.index:
            in_string = df.loc[i,'text']
            df.loc[i,'text'] = lemmatize_sentence(in_string)
    except Exception as inst:
        print(type(inst))  # the exception type
        print(inst.args)  # arguments stored in .args
        print(inst)         # __str__ allows args to be printed directly,
                        # but may be overridden in exception subclasses
        #print(count)
    return df

### Remove punctuation

In [17]:
def remove_punctuation(df):
    df['text'] = df['text'].apply(lambda text: ' '.join(filter(lambda new_word: new_word != '', 
                                                               [re.sub(r'[^\w\s]', '', word) for word in text.split()])))
    return df

## Preprocess

In [7]:
def normalize(df_in):
    df = remove_non_ascii(df_in)
    df = to_lowercase(df)
    df = remove_punctuation(df)
    df = replace_numbers(df)
    #df = remove_stopwords(df)
    #df = lemmatize(df)
    return df

In [6]:
def preprocess(df):
    updated_df = remove_URL(df)
    updated_df = replace_contractions(updated_df)
    # Tokenize
    #words = nltk.word_tokenize(sample)
    normalized_df = normalize(updated_df)
    # Normalize
    return normalized_df

In [21]:
df.head(10)

,text,label
0,Ottawa police are confirming a shooting at the...,non-rumor
1,Passersby working on shooting victim. From the...,non-rumor
2,Shooting at the war memorial in Ottawa. http:/...,non-rumor
3,After today we will officially be done shootin...,non-rumor
4,KEY: Nothing in Wilson's story in new autopsy ...,non-rumor
5,Ottawa police confirm a shooting at Canada's N...,non-rumor
6,#BREAKING: Shooting reported at the War Memori...,non-rumor
7,#BREAKINGNEWS: Update - One person shot at War...,non-rumor
8,The scene at the War Memorial in Ottawa http:/...,non-rumor
9,Ottawa police tactical officers are here and v...,non-rumor


In [65]:
rumor_data = preprocess(r_data)
print(rumor_data.head())

                                                text  label
0  recap gunman shot dead inside parliament build...  rumor
1  soldier killed at war memorial identified as n...  rumor
2  soldier killed at war memorial identified as c...  rumor
3  breaking two or three gunmen were involved in ...  rumor
4  all three patients injured in ottawashooting r...  rumor


In [67]:
rumor_data.to_csv('preprocessed_rumor_data.csv', index=False)

In [66]:
non_rumor_data = preprocess(nr_data)
print(non_rumor_data.head())

                                                text      label
0  ottawa police are confirming a shooting at the...  non-rumor
1  passersby working on shooting victim from the ...  non-rumor
2  shooting at the war memorial in ottawa shootin...  non-rumor
3  after today we will officially be done shootin...  non-rumor
4  key nothing in wilsons story in new autopsy sa...  non-rumor


In [68]:
non_rumor_data.to_csv('preprocessed_non_rumor_data.csv', index=False)

In [25]:
preprocessed_df = preprocess(df)
print(preprocessed_df.head())

                                                text      label
0  ottawa police are confirming a shooting at the...  non-rumor
1  passersby working on shooting victim from the ...  non-rumor
2  shooting at the war memorial in ottawa shootin...  non-rumor
3  after today we will officially be done shootin...  non-rumor
4  key nothing in wilsons story in new autopsy sa...  non-rumor


In [26]:
preprocessed_df.head()

,text,label
0,ottawa police are confirming a shooting at the...,non-rumor
1,passersby working on shooting victim from the ...,non-rumor
2,shooting at the war memorial in ottawa shootin...,non-rumor
3,after today we will officially be done shootin...,non-rumor
4,key nothing in wilsons story in new autopsy sa...,non-rumor


In [60]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    890 non-null    object
 1   label   890 non-null    object
dtypes: object(2)
memory usage: 14.0+ KB


In [27]:
preprocessed_df.to_csv('preprocessed_data.csv', index=False)